In [1]:
import numpy as np
from skimage.measure import marching_cubes
from scipy.interpolate import RegularGridInterpolator
from scipy.spatial import cKDTree

# ---------------- Runtime options ----------------
VERBOSE = False   # set True for diagnostics

# ---------------- Cube file reader ---------------
def read_cube_file(filename, label="CUBE"):
    with open(filename, "r") as f:
        lines = f.readlines()

    # First two lines are comments
    header = lines[2].split()
    natoms = int(header[0])
    origin = np.array(header[1:4], dtype=float)

    # Grid vectors (number of points + vector components)
    nx, *vx = lines[3].split()
    ny, *vy = lines[4].split()
    nz, *vz = lines[5].split()
    nx, ny, nz = int(nx), int(ny), int(nz)
    vx, vy, vz = np.array(vx, dtype=float), np.array(vy, dtype=float), np.array(vz, dtype=float)

    # Skip atom lines
    data_start = 6 + natoms

    # Read volumetric data
    values = []
    for line in lines[data_start:]:
        values.extend(map(float, line.split()))
    data = np.array(values)

    if data.size != nx * ny * nz:
        raise ValueError(f"Cube size mismatch: expected {nx*ny*nz}, got {data.size}")

    # ORCA: Z fastest, then Y, then X
    data = data.reshape((nx, ny, nz), order="C")
    grid_vectors = np.vstack([vx, vy, vz])
    
    #---------------- Error testing ---------------
    if VERBOSE:
        print(f"\n[{label}] Volumetric data statistics")
        print(f"  min  : {np.min(data):.6f}")
        print(f"  max  : {np.max(data):.6f}")
        print(f"  mean : {np.mean(data):.6f}")
    #----------------------------------------------
    return data, origin, grid_vectors

# ---------------- PQR writer ----------------
def write_pqr_file(filename, vertices, esp_values, label="UNK"):
    """Writes a PQR file using strict column widths for charged molecules."""
    with open(filename, 'w') as f:
        for i, (xyz, esp) in enumerate(zip(vertices, esp_values)):
            q = esp if not np.isnan(esp) else 0.0
            
            # Use 8.4 for standard values, 8.3 if values are very large/negative
            # to maintain strict 8-character width for the 'charge' column
            q_fmt = f"{q:8.3f}" if (q <= -10.0 or q >= 100.0) else f"{q:8.4f}"
            
            # PQR Column format: Field, ID, Name, Res, Chain, ResID, X, Y, Z, Q, R
            line = (
                f"ATOM  {(i+1)%100000:>5d}  H   {label:>3} X   1    "
                f"{xyz[0]:8.3f}{xyz[1]:8.3f}{xyz[2]:8.3f}"
                f"{q_fmt}  1.500\n"
            )
            f.write(line)    
    
# ---------------- Cube → surface generator ----------------
def generate_surface_from_cube(eldens_file, esp_file, isovalue=0.001):
    eldens_data, origin, grid_vectors = read_cube_file(
        eldens_file,
        label="DENSITY CUBE"
    )

    esp_data, esp_origin, esp_vectors = read_cube_file(
        esp_file,
        label="ESP CUBE"
    )

    if not np.allclose(origin, esp_origin) or not np.allclose(grid_vectors, esp_vectors):
        raise ValueError("ELDENS and ESP cube grids do not match")

    # Bohr → Å
    BOHR_TO_ANG = 0.52917721092
    origin *= BOHR_TO_ANG
    grid_vectors *= BOHR_TO_ANG

    # Marching cubes (Z,Y,X)
    eldens_mc = np.transpose(eldens_data, (2,1,0))
    verts_mc, faces, _, _ = marching_cubes(eldens_mc, level=isovalue)

    # Convert voxel indices → real space
    verts_voxel = verts_mc[:, [2,1,0]]
    verts_real = verts_voxel @ grid_vectors + origin

    # ESP interpolation
    nx, ny, nz = esp_data.shape
    interpolator = RegularGridInterpolator(
        (np.arange(nx), np.arange(ny), np.arange(nz)),
        esp_data,
        bounds_error=False,
        fill_value=None
    )

    voxel_coords = np.linalg.solve(
        grid_vectors.T,
        (verts_real - origin).T
    ).T

    esp_vals = interpolator(voxel_coords)
    
    #---------------- Error testing ---------------
    print("\n[SURFACE]")
    print(f"  electron density isovalue : {isovalue:.6f} a.u.")
    print(f"  vertices                  : {len(verts_real)}")

    if VERBOSE:
        inside = (
            not np.any(voxel_coords < 0) and
            not np.any(voxel_coords > [nx-1, ny-1, nz-1])
        )

        status = "OK" if inside else "WARNING"
        print(f"  interpolation inside grid : {status}")

    #----------------------------------------------
    
    return verts_real, esp_vals


# ---------------- PQR reader ----------------
def read_pqr(filename):
    """Reads PQR using string slicing (safe for merged columns)."""
    coords, esp = [], []
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                try:
                    # Slicing is safer than .split() for charged molecules
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                    q = float(line[54:62])
                    coords.append([x, y, z])
                    esp.append(q)
                except (ValueError, IndexError):
                    continue
    return np.array(coords), np.array(esp)
# ---------------- Local extrema ----------------
def find_local_extrema(coords, values, radius=0.2, mode="max", tol=1e-4):
    """Find local maxima or minima in a neighborhood."""
    tree = cKDTree(coords)
    extrema = []
    for i, r in enumerate(coords):
        idx = tree.query_ball_point(r, radius)
        if len(idx) < 5:  # skip isolated points
            continue
        neighborhood = values[idx]
        if mode == "max" and values[i] >= neighborhood.max() - tol:
            extrema.append(i)
        elif mode == "min" and values[i] <= neighborhood.min() + tol:
            extrema.append(i)
    return np.array(extrema)

# ---------------- Sigma-hole detection ----------------
def sigma_hole_candidates(coords, esp, center, direction, max_angle_deg=40, min_distance=0.2):
    rvecs = coords - center
    distances = np.linalg.norm(rvecs, axis=1)
    mask = distances > min_distance
    rvecs, esp_sel = rvecs[mask], esp[mask]
    rhat = rvecs / np.linalg.norm(rvecs, axis=1)[:,None]
    cosang = np.dot(rhat, direction)
    angle_mask = cosang > np.cos(np.deg2rad(max_angle_deg))
    #---------------- Error testing --------------------
    if VERBOSE:
        print(f"[σ-hole] candidates after filtering: {np.sum(angle_mask)}")
    #---------------------------------------------------
    return coords[mask][angle_mask], esp_sel[angle_mask]

def compute_dVsigma(coords, esp, center, direction,
                    sigma_angle_deg=40,
                    ring_inner_deg=50,
                    ring_outer_deg=90,
                    min_distance=0.2):

    rvecs = coords - center
    distances = np.linalg.norm(rvecs, axis=1)

    mask = distances > min_distance
    rvecs = rvecs[mask]
    esp_sel = esp[mask]

    rhat = rvecs / np.linalg.norm(rvecs, axis=1)[:, None]
    cosang = np.dot(rhat, direction)
    angles = np.degrees(np.arccos(np.clip(cosang, -1, 1)))

    # sigma cone
    sigma_mask = angles <= sigma_angle_deg

    # surrounding ring
    ring_mask = (angles >= ring_inner_deg) & (angles <= ring_outer_deg)

    if not np.any(sigma_mask) or not np.any(ring_mask):
        return None, None, None

    V_sigma = np.max(esp_sel[sigma_mask])
    V_ring = np.mean(esp_sel[ring_mask])

    dVsigma = V_sigma - V_ring

    return V_sigma, V_ring, dVsigma

    
def detect_sigma_holes(coords, esp, halogen_positions, bonded_atom_positions,
                       max_angle_deg=40, min_distance=0.2):
    sigma_holes = []
    for X, A in zip(halogen_positions, bonded_atom_positions):
        bond_vec = X - A
        bond_vec /= np.linalg.norm(bond_vec)
        if VERBOSE:
            print(f"[σ-hole] axis: {bond_vec}")
        sigma_coords, sigma_esp = sigma_hole_candidates(
            coords, esp, center=X, direction=bond_vec,
            max_angle_deg=max_angle_deg, min_distance=min_distance
        )
        if len(sigma_esp) == 0: continue
        max_idx = sigma_esp.argmax()
        V_sigma, V_ring, dVsigma = compute_dVsigma(
            coords, esp,
            center=X,
            direction=bond_vec,
            sigma_angle_deg=max_angle_deg,
            ring_inner_deg=max_angle_deg + 10,
            ring_outer_deg=max_angle_deg + 50,
            min_distance=min_distance
        )
        
        sigma_holes.append({
            "halogen_pos": X,
            "bond_vec": bond_vec,
            "sigma_hole_value": sigma_esp[max_idx],
            "sigma_hole_pos": sigma_coords[max_idx],
            "V_ring_avg": V_ring,
            "dVsigma": dVsigma
        })
    return sigma_holes

# ---------------- Write extrema + sigma holes ----------------
def write_extrema_pqr(filename, coords, esp, local_max_idx, local_min_idx, sigma_holes):
    with open(filename, "w") as f:
        atom_id = 0

        # maxima
        for i in local_max_idx:
            x, y, z = coords[i]
            q = esp[i]
            f.write(
                f"ATOM  {atom_id:5d}  H   MAX X   1    "
                f"{x:8.3f}{y:8.3f}{z:8.3f}{q:10.6f}  1.50\n"
            )
            atom_id += 1

        # minima
        for i in local_min_idx:
            x, y, z = coords[i]
            q = esp[i]
            f.write(
                f"ATOM  {atom_id:5d}  H   MIN X   1    "
                f"{x:8.3f}{y:8.3f}{z:8.3f}{q:10.6f}  1.50\n"
            )
            atom_id += 1

        # sigma holes
        for sh in sigma_holes:
            x, y, z = sh["sigma_hole_pos"]
            q = sh["sigma_hole_value"]
            f.write(
                f"ATOM  {atom_id:5d}  H   SIG X   1    "
                f"{x:8.3f}{y:8.3f}{z:8.3f}{q:10.6f}  1.50\n"
            )
            atom_id += 1

    print(f"  Extrema PQR written → {filename}")
    

if __name__ == "__main__":
    # 0. Ask user for input files and parameters
    eldens_file = input("Enter ELDENS cube file name (e.g., 4Fe.eldens.cube): ").strip()
    esp_file = input("Enter ESP cube file name (e.g., 4Fe.scfp.esp.cube): ").strip()
    
    try:
        isovalue = float(input("Enter isosurface value for electron density (e.g., 0.001): ").strip())
    except ValueError:
        print("Invalid isovalue, using default 0.001")
        isovalue = 0.001

    try:
        extrema_radius = float(input("Enter radius (Å) for local extrema detection (e.g., 0.2): ").strip())
    except ValueError:
        print("Invalid radius, using default 0.2 Å")
        extrema_radius = 0.2
        
    try:
        min_distance_entry = float(input("Enter the minimal distance from Halogen atom (Å) for local extrema detection (e.g., 0.2): ").strip())
    except ValueError:
        print("Invalid distance, using default 0.2 Å")
        min_distance_entry = 0.2
    
    try:
        max_angle = float(input("Enter the maximal angle of the cone along C-X bond for sigma hole detection (e.g., 15°): ").strip())
    except ValueError:
        print("Invalid cone angle, using default of 15°")
        max_angle = 15
        
        

    # 1. Generate isosurface directly in memory
    coords, esp = generate_surface_from_cube(
        eldens_file,
        esp_file,
        isovalue=isovalue
    )
    
    #------- Surface Statistics and Errors --------
    print("\n[ESP ON SURFACE]")
    print(f"  range : {np.nanmin(esp):.5f} → {np.nanmax(esp):.5f}")
    if VERBOSE:
        print(f"  mean  : {np.mean(esp):.5f}")
        print(f"  std   : {np.std(esp):.5f}")
        outliers = np.abs(esp) > 50
        print(f"  extreme points (>50 a.u.): {np.sum(outliers)}")
    #----------------------------------------------    
    
    # 2. Find local extrema
    local_max_idx = find_local_extrema(coords, esp, radius=extrema_radius, mode="max", tol=1e-4)
    local_min_idx = find_local_extrema(coords, esp, radius=extrema_radius, mode="min", tol=1e-4)

        
    # 3. Define halogen positions / bonded atoms (example, replace with your real coords)
    #--------------ADD YOUR COORDINATES HERE -------
    halogen_positions = np.array([[-3.847, -3.323, -2.625],[ -3.642, 3.377, 2.721]])
    bonded_atom_positions = np.array([[-4.170, -4.011, -0.881],[-4.116, 3.960, 0.973]])

    # 4. Detect sigma holes
    sigma_holes = detect_sigma_holes(
        coords, esp,
        halogen_positions,
        bonded_atom_positions,
        max_angle_deg=max_angle,
        min_distance=min_distance_entry
    )

    # 5. Print sigma hole strengths
    print("\n[σ-HOLE ANALYSIS]")

    if not sigma_holes:
        print("  No sigma holes detected.")
    else:
        for i, sh in enumerate(sigma_holes, 1):
            pos = sh["sigma_hole_pos"]

            if sh["dVsigma"] is None:
                print(
                    f"  #{i}  Vσ = {sh['sigma_hole_value']:.4f} a.u."
                    f"  @ ({pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f})"
                )
            else:
                print(
                    f"  #{i}  Vσ = {sh['sigma_hole_value']:.4f} a.u."
                    f"  dVσ = {sh['dVsigma']:.4f} a.u."
                    f"  @ ({pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f})"
                )

    # 6. Optional: export full surface
    write_pqr_file("sigma_surface.pqr", coords, esp)
    print("\n[OUTPUT]")
    print("  sigma_surface.pqr  — ESP mapped surface")

   

    # 7. Write extrema + sigma holes to PQR
    write_extrema_pqr(
        "extrema.pqr",
        coords,
        esp,
        local_max_idx,
        local_min_idx,
        sigma_holes
    )

Enter ELDENS cube file name (e.g., 4Fe.eldens.cube): 4Fe.eldens.cube
Enter ESP cube file name (e.g., 4Fe.scfp.esp.cube): 4Fe.scfp.esp.cube
Enter isosurface value for electron density (e.g., 0.001): 0.001
Enter radius (Å) for local extrema detection (e.g., 0.2): 0.1
Enter the minimal distance from Halogen atom (Å) for local extrema detection (e.g., 0.2): 0.2
Enter the maximal angle of the cone along C-X bond for sigma hole detection (e.g., 15°): 40

[DENSITY CUBE] Volumetric data statistics
  min  : 0.000000
  max  : 449.993000
  mean : 0.013340

[ESP CUBE] Volumetric data statistics
  min  : -0.309862
  max  : 300.685000
  mean : -0.076320

[SURFACE]
  electron density isovalue : 0.001000 a.u.
  vertices                  : 139994
  interpolation inside grid : OK

[ESP ON SURFACE]
  range : -0.28599 → -0.07799
  mean  : -0.14892
  std   : 0.04467
  extreme points (>50 a.u.): 0
[σ-hole] axis: [ 0.16978358  0.36164429 -0.91672621]
[σ-hole] candidates after filtering: 975
[σ-hole] axis: [ 